In [ ]:
%matplotlib inline
from __future__ import print_function
import gc
import ipywidgets
import math
import os
import random
import sys
import traceback
import sklearn.metrics

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from IPython.display import Image
from IPython.display import display
from scipy import ndimage

import outputer
import improc
import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (improc)
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

# Enumerate Images
Image names are sequential, so add every tenth image to the validation set based on filename.

In [ ]:
training = []
test = []

for root, dirs, files in os.walk('captures'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        # Find all the image files, split into test and training.
        if low_name.endswith(".png"):
            if low_name.endswith("0.png"):
                test.append(path)
            else:
                training.append(path)

print("Training:", len(training), "Test:", len(test))
print(training[:2])
print(test[:2])

# Image Processing
Each image file contains a color image (top half), and an encoded depth image (bottom half)
<img src="testing/IMG_2114.PNG">
* Note: The image may also contain the orientation data. If so it is encoded in the first two pixels of the depth image. If the first pixel of the depth image is red, the second has the x, y, z, w quaternion components encoded in the r,g,b,a values.

The improc module contains functions for splitting the image, decoding the depth back into floating point millimeters, and for filling in gaps.

In [ ]:
# Precomputed via compute_average_depth()
MEAN_DEPTH = np.float32(1688.97)

print(MEAN_DEPTH)

In [ ]:
depth_image_cache_path = outputer.setup_directory("temp", "cache")

class ImageSampler(object):
    """Wrap an image for sampling."""
    def __init__(self, image_file):
        # Process the image or grab it from the cache.
        # image is normalized CIELAB, depth is not normalized.
        self.image, self.depth = improc.process_cached(depth_image_cache_path, image_file)
        self.depth /= improc.MAX_DEPTH

    def sample(self, image_slot, depth_slot, height_offset=None, width_offset=None):
        height = image_slot.shape[0]
        spare_height = self.image.shape[0] - height
        y = spare_height / 2 if height_offset is None else height_offset
        
        width = image_slot.shape[1]
        spare_width = self.image.shape[1] - width
        x = spare_width / 2 if width_offset is None else width_offset
        
        image_slot[:,:,:] = self.image[y : y + height, x : x + width, : image_slot.shape[-1]]
        depth_slot[:,:,0] = self.depth[y : y + height, x : x + width]

In [ ]:
def prepare_images(image_paths, inputs, targets):
    for i, sampler in enumerate([ImageSampler(path) for path in image_paths]):
        sampler.sample(inputs[i], targets[i])

## Image processing examples

In [ ]:
example_image, example_depth, example_attitude = improc.load_image("testing/IMG_2114.PNG")
plt.imshow(example_image)
print(example_image.shape, example_image.dtype)

In [ ]:
plt.imshow(example_depth)
print(example_depth.shape, example_depth.dtype)
print(example_attitude)

In [ ]:
sampler = ImageSampler("testing/IMG_2114.PNG")
sample_size = 100
image_sample = np.zeros(shape=(sample_size, sample_size, 3))
depth_sample = np.zeros(shape=(sample_size, sample_size, 1))
sampler.sample(image_sample, depth_sample)

print(image_sample.shape, image_sample.dtype)
plt.imshow(image_sample)

In [ ]:
plt.imshow(depth_sample.reshape(sample_size, sample_size))
print(depth_sample.shape, depth_sample.dtype)
print(np.min(depth_sample), np.max(depth_sample))

# Data Management

In [ ]:
COLOR_CHANNELS = 3
image_height = 480
image_width = 640

data_files = {
    "image_size": (image_height, image_width, COLOR_CHANNELS),
    "depth_size": (image_height, image_width, 1),
    "train_files": np.array(sorted(training)),
    "test_files": np.array(sorted(test))
}

del training
del test

In [ ]:
def setup_cross_validation(data, valid_count, test_count=None, chunk_size=None, entropy=random):
    cross_data = data.copy()
    
    if chunk_size:
        cross_data["image_size"] = chunk_size
        cross_data["depth_size"] = chunk_size[:-1] + (1,)

    paths = cross_data["train_files"][:]
    mutate.fisher_yates_shuffle(paths, entropy)

    cross_data["train_files"] = paths[:-valid_count]
    cross_data["valid_files"] = paths[-valid_count:]
    
    if test_count is None:
        del cross_data["test_files"]
    else:
        cross_data["test_files"] = data["test_files"][:test_count]
    
    return cross_data

# Graph Setup

In [ ]:
def setup_graph(
    batch_size,
    image_shape,
    target_shape,
    layer_stack,
    include_coords=True
):
    graph = tf.Graph()
    with graph.as_default():
        input_shape = (batch_size,) + image_shape
        output_shape = (batch_size,) + target_shape
        train   = tf.placeholder(tf.float32, shape=input_shape)
        targets = tf.placeholder(tf.float32, shape=output_shape)
        verify  = tf.placeholder(tf.float32, shape=input_shape)
        
        if include_coords:
            y_size = image_shape[0]
            x_size = image_shape[1]
            coords = np.zeros(shape=(batch_size, y_size, x_size, 2), dtype=np.float32)
            coords[:, :, :, 0] = np.linspace(-1, 1, num=y_size, dtype=np.float32).reshape((1, y_size, 1))
            coords[:, :, :, 1] = np.linspace(-1, 1, num=x_size, dtype=np.float32).reshape((1, 1, x_size))
            input_shape = input_shape[:-1] + (input_shape[-1] + coords.shape[-1],)
            train = tf.concat(3, [train, tf.constant(coords)])
            verify = tf.concat(3, [verify, coords])
        
        layers = layer_stack.construct(input_shape, output_shape)
        l2_loss = convnet.setup_layers(layers)
        
        results = convnet.connect_model(train, layers, True)[-1]
        
        # Fill NaNs in target with values from results to
        # eliminate any contribution to the gradient
        valid_targets = tf.select(tf.is_nan(targets), results, targets)
        
        loss = tf.reduce_mean(tf.squared_difference(results, valid_targets)) + l2_loss
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "targets": targets,
            "loss": loss,
            "optimizer": layer_stack.construct_optimizer(loss),

            # Predictions for training and verification (validation or test)
            "predictions": results,
            "verify": verify,
            "verify_predictions": convnet.connect_model(verify, layers, False)[-1],
            "saver": tf.train.Saver()
        }
    return info

# Graph Execution

In [ ]:
def prediction_error(predictions, targets):
    is_finite = np.isfinite(targets)
    where_valid = np.where(is_finite)
    error = sklearn.metrics.mean_squared_error(predictions[where_valid], targets[where_valid])
    return error, np.count_nonzero(is_finite)

In [ ]:
def make_predictor(session, graph_info):
    def predict(inputs, targets):
        feed_dict = {graph_info["verify"]: inputs}
        return session.run([graph_info["verify_predictions"]], feed_dict=feed_dict)[0]
    return predict

In [ ]:
def batch_prediction_error(predictor, files, inputs, targets, batch_size):
    total_error = 0
    total_count = 0
    batch_count = len(files) / batch_size
    for b in xrange(batch_count):
        offset = b * batch_size
        end = offset + batch_size
        prepare_images(files[offset:end], inputs, targets)
        predictions = predictor(inputs, targets)
        error, count = prediction_error(predictions, targets)
        total_error += error * count
        total_count += count
    return (total_error / np.float32(total_count)), predictions[-1], targets[-1]

In [ ]:
def depth_mean_like(depths):
    return np.ones_like(depths) * (MEAN_DEPTH / improc.MAX_DEPTH)

def always_guess_mean_error(files, inputs, targets, batch_size):
    def predict_mean(images, depths):
        return depth_mean_like(depths)
    return batch_prediction_error(predict_mean, files, inputs, targets, batch_size)[0]

In [ ]:
def batch_input_shape(batch_size, data):
    return (batch_size,) + data["image_size"]

def batch_output_shape(batch_size, data):
    return (batch_size,) + data["depth_size"]

def score_run(guess_mean_error, valid_error):
    return guess_mean_error - min(valid_error, 1)

def run_graph(
    graph_info,
    data,
    step_count,
    report_every=50,
    verbose=True,
    progress=None,
    tracker=None,
    mean_error_cache=None,
    error_maximum=None
):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")

        # Optionally restore graph parameters from disk.
        convnet.restore_model(graph_info, session)
        
        batch_size = graph_info["batch_size"]
        batch_inputs = np.empty(shape=batch_input_shape(batch_size, data), dtype=np.float32)
        batch_targets = np.empty(shape=batch_output_shape(batch_size, data), dtype=np.float32)

        # Validation and scoring bits.
        valid_error = 1
        
        guess_mean_error = None
        if mean_error_cache is not None:
            guess_mean_error = mean_error_cache.get("cached")
        if not guess_mean_error:
            guess_mean_error = always_guess_mean_error(
                data["valid_files"], batch_inputs, batch_targets, batch_size
            )
            print("Error if just guess mean:", guess_mean_error)
            
            if mean_error_cache is not None:
                mean_error_cache["cached"] = guess_mean_error
        predictor = make_predictor(session, graph_info)
        
        training_files = data["train_files"]
        try:
            for step in xrange(step_count + 1):
                # Update progress bar, if present
                if progress:
                    progress.value = step

                # Generate a minibatch.
                offset = (step * batch_size) % (training_files.shape[0] - batch_size)
                batch_files = training_files[offset:(offset + batch_size)]
                prepare_images(batch_files, batch_inputs, batch_targets)

                # Graph evaluation targets:
                targets = [
                    graph_info["optimizer"],
                    graph_info["loss"],
                    graph_info["predictions"]
                ]
                
                # Graph inputs:
                feed_dict = {
                    graph_info["train"] : batch_inputs,
                    graph_info["targets"] : batch_targets
                }
                
                # Run the graph
                _, loss, predictions = session.run(targets, feed_dict=feed_dict)
                
                # Capture last prediction
                results = (predictions[-1], batch_targets[-1], batch_inputs[-1])
                
                # Update stats:
                reporting = step % report_every == 0
                if reporting or tracker:
                    batch_error, _ = prediction_error(predictions, batch_targets)
                    if tracker:
                        tracker((loss, batch_error))
                
                if not np.isfinite(loss):
                    print("Error computing loss:", loss)
                    print(np.sum(np.isnan(predictions)))
                    return score_run(guess_mean_error, valid_error), results
                
                if reporting:
                    if verbose:
                        print("Minibatch loss at step", step, ":", loss)
                        print("Minibatch error:", batch_error)
                    valid_error, _, _ = batch_prediction_error(
                        predictor, data["valid_files"],
                        batch_inputs, batch_targets, batch_size
                    )
                    print("Validation error:", valid_error)
                    if error_maximum and step > 0 and valid_error < error_maximum:
                        print("Early out.")
                        break
            test_files = data.get("test_files")
            if test_files is not None:
                test_results = batch_prediction_error(
                    predictor, test_files,
                    batch_inputs, batch_targets, batch_size
                )
                print("Test error:", test_results[0])
                results = results + test_results
            return score_run(guess_mean_error, valid_error), results
        finally:
            # Optionally save out graph parameters to disk.
            convnet.save_model(graph_info, session)

## Testing Error Metrics

In [ ]:
TEST_BATCH = 1
test_inputs = np.empty(shape=(TEST_BATCH, 480, 640, COLOR_CHANNELS), dtype=np.float32)
test_depths = np.empty_like(test_inputs[:,:,:,:1])
prepare_images(data_files["test_files"][:TEST_BATCH], test_inputs, test_depths)

In [ ]:
plt.imshow(test_inputs[0,:,:,0], cmap='Greys_r')
print(np.min(test_inputs[0,:,:,0]),np.max(test_inputs[0,:,:,0]))
print(np.min(test_inputs[0,:,:,1]),np.max(test_inputs[0,:,:,1]))
print(np.min(test_inputs[0,:,:,2]),np.max(test_inputs[0,:,:,2]))

In [ ]:
plt.imshow(test_depths[0,:,:,0])
depths_valid = np.where(np.isfinite(test_depths[0]))
print(np.min(test_depths[0][depths_valid]),np.max(test_depths[0][depths_valid]))

In [ ]:
print(test_inputs.shape)
print(test_depths.shape)
prediction_error(test_inputs[:,:,:,0:1], test_depths)

In [ ]:
prediction_error(np.zeros_like(test_depths), test_depths)

In [ ]:
prediction_error(depth_mean_like(test_depths), test_depths)

## Testing components

In [ ]:
TEST_BATCH = 1
conv_layers = [
    ("conv",       5, 2, 10, "SAME", False),
    ("conv",      10, 2, 20, "SAME", False),
    ("conv_bias", 15, 5, 25, "SAME", False)
]
expand_layers = [
    (5, 5, "SAME", True, False),
    (2, 5, "SAME", True, False),
    (2, 5, "SAME", True, False)
]
test_stack = convevo.create_stack(conv_layers, expand_layers, False, [], 0.0, 0.01, 0.0)
test_stack.make_safe((TEST_BATCH,) + data_files["image_size"], (TEST_BATCH,) + data_files["depth_size"])
test_stack.reseed(random.Random(24601))

In [ ]:
sample_size = data_files["image_size"]
depth_size = data_files["depth_size"]

test_graph = setup_graph(TEST_BATCH, sample_size, depth_size, test_stack)

In [ ]:
test_cross = setup_cross_validation(data_files, 200, 200, sample_size)
test_score, test_results = run_graph(test_graph, test_cross, 8, 4, True)
print(score)

In [ ]:
plt.imshow(test_results[0][:,:,0])
print(np.min(test_results[0]),np.max(test_results[0]))

In [ ]:
plt.imshow(test_results[1].reshape(sample_size[0],sample_size[1]))

In [ ]:
plt.imshow(test_results[2][:,:,0], cmap='Greys_r')

In [ ]:
del test_stack
del test_graph
del test_cross
del results
gc.collect()

# Evolving

In [ ]:
results_path = outputer.setup_directory("temp", "pyndent_results")

def save_results(timestamp, results):
    with open(os.path.join(results_path, timestamp + ".csv"), "w") as text_file:
        text_file.write("Loss,Depth Error\n")
        for score in results:
            text_file.write((",".join(str(v) for v in score)) + "\n")
    print("Saved results:", timestamp)

In [ ]:
def make_eval(batch_size, eval_steps, valid_size, reuse_cross, entropy=random):
    mean_error_cache = None
    if reuse_cross:
        redata = setup_cross_validation(
            data_files, valid_size, entropy=entropy
        )
        mean_error_cache = {}

    # Set up to show a progress bar so you some mesure of time required. Updated in run_graph above.
    progress_bar = ipywidgets.FloatProgress(min=0, max=eval_steps, description="Graph Steps:")
    display(progress_bar)
    
    # Set up to show current training results as well as a running average. updated in record_score below. 
    def setup_label(title):
        return ipywidgets.FloatText(value=0, description=title, disabled=True)
    current_display = [setup_label(title) for title in ["Loss", "Error"]]
    average_display = [setup_label(" ") for _ in current_display]
    display(ipywidgets.HBox([
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Current</div>")] + current_display),
        ipywidgets.Box([ipywidgets.HTML("<div style=""margin-left:90px"">Running Average</div>")] + average_display)
    ]))
        
    def evaluate(stack, eval_entropy):
        # If not reusing data, generate training and validation sets
        if not reuse_cross:
            data = setup_cross_validation(
                data_files, valid_size, entropy=eval_entropy
            )
        else:
            data = redata

        # Set up the Tensorflow graph
        try:
            evo_graph = setup_graph(batch_size, data["image_size"], data["depth_size"], stack)
        except KeyboardInterrupt:
            raise
        except:
            # Record any errors and the stack that caused them.
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, "temp", outputer.timestamp("ERR~", "txt"))
            return -10

        timestamp = outputer.timestamp()
        with open(os.path.join(results_path, timestamp + ".xml"), "w") as text_file:
            text_file.write(convevo.serialize(stack))
    
        convnet.setup_save_model(evo_graph, os.path.join(results_path, timestamp + ".ckpt"))
        
        # Record and display the results
        results = []
        def record_score(score):
            results.append(score)
            for display, value in zip(current_display, score):
                display.value = value
            
            resultCount = min(len(results), 100)
            averages = [sum(x)/resultCount for x in zip(*results[-resultCount:])]
            for display, value in zip(average_display, averages):
                display.value = value
                
        # Run the graph
        try:
            valid_error, _ = run_graph(
                evo_graph,
                data,
                eval_steps,
                report_every=eval_steps/4,
                verbose=True,
                progress=progress_bar,
                tracker=record_score,
                mean_error_cache=mean_error_cache
            )
            return valid_error
        except KeyboardInterrupt:
            raise
        except:
            # Record any errors and the stack that caused them.
            exc_type, exc_value, exc_traceback = sys.exc_info()
            lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
            print(lines[-1])
            convevo.output_error(stack, lines, results_path, timestamp + "~ERR.txt")
            return -1
        finally:
            save_results(timestamp, results)
    return evaluate

In [ ]:
conv_layers = [
    ("conv",       5, 2, 10, "SAME", False),
    ("conv",      10, 2, 20, "SAME", False),
    ("conv_bias", 15, 5, 25, "SAME", False)
]
expand_layers = [
    (5, 5, "SAME", True, False),
    (2, 5, "SAME", True, False),
    (2, 5, "SAME", True, False)
]
prototype = convevo.create_stack(conv_layers, expand_layers, False, [], 0.0, 0.01, 0.0)

prototypes = [prototype]

In [ ]:
population,_,_ = convevo.load_population("temp/2016-06-07~15_58_01_762.xml", False)
prototypes = population[:5]
print(len(prototypes))

In [ ]:
prototypes = [
    convevo.load_stack("testing/pyndent1.xml"),
    convevo.load_stack("testing/pyndent2.xml"),
    convevo.load_stack("testing/pyndent3.xml")
]

In [ ]:
prototypes = [
    convevo.load_stack("testing/pyndent4.xml")
]
print(convevo.serialize(prototypes[0]))

In [ ]:
with outputer.TeeOutput(os.path.join("temp", outputer.timestamp("Pyndent_Evolve_", "txt"))):
    mutate_seed = random.randint(1, 100000)
    print("Mutate Seed:", mutate_seed)
    mutate_entropy = random.Random(mutate_seed)
    eval_seed = random.randint(1, 100000)
    print("Eval Seed:", eval_seed)
    eval_entropy = random.Random(eval_seed)

    population_size = 5
    generations = 2
    batch_size = 1

    breed_options = {
        "input_shape": batch_input_shape(batch_size, data_files),
        "output_shape": batch_output_shape(batch_size, data_files)
    }

    for stack in prototypes:
        stack.make_safe(breed_options["input_shape"], breed_options["output_shape"])

    evaluator = make_eval(
        batch_size=batch_size, eval_steps=80000, valid_size=400,
        reuse_cross=True, entropy=eval_entropy
    )
    charles = darwin.Darwin(convevo.serialize, evaluator, convevo.breed)
    charles.init_population(prototypes, population_size, True, breed_options, mutate_entropy)

    for g in range(generations):
        print("Generation", g)
        results = charles.evaluate(eval_entropy)
        convevo.output_results(results, "temp", outputer.timestamp() + ".xml", mutate_seed, eval_seed)
        charles.repopulate(population_size, 0.3, 3, results, breed_options, mutate_entropy)

In [ ]:
results = darwin.descending_score(charles.history.values())
convevo.output_results(results, "testing", "pyndent_candidate4_evolve.xml", mutate_seed, eval_seed)
len(results)